# Pixelbuffer Identity Graph

A graph that accepts a pixel buffer normalized to `[0,1]` values and transforms the output to a pixel buffer normalized to values in the range `[-1,1]`. The purpose of this model is to verify the normalization and denormalization functions.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
X = tf.placeholder(tf.float32, shape=(224,224,3), name="input")

Z = tf.add(tf.multiply(X, 2), -1, name="output")

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([Z], feed_dict={
        X: np.full([224,224,3], 0.5, dtype=np.float32)
    })
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input": X}, outputs={"output": Z})
    
    # print(z_eval)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input \
	--input_shapes=224,224,3 \
	--output_arrays=output
```